PYTHON-12. Работа с файлами разных форматов

# START

In [ ]:
import warnings; warnings.simplefilter('ignore')

import random
import collections

import pandas as pd
import numpy as np
import os
import re

import json  
from pprint import pprint  

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


%matplotlib inline


In [ ]:
project_dir = r'/content/drive/My Drive/Colab Notebooks/data/u3/'
os.listdir(project_dir)

## 12.1 Формат JSON

In [ ]:
with open(project_dir + 'recipes.json') as f:
   recipes = json.load(f)

In [ ]:
pprint(recipes[0:3])

In [ ]:
len(recipes), type(recipes)

In [ ]:
recipes[0]['id']

In [ ]:
recipes[0]['ingredients']

In [ ]:
find_id = 13121

for recipe in recipes:
    if recipe['id'] == find_id:
        print(recipe['cuisine'])
        break

In [ ]:
# Task 12.1.1

recipes[len(recipes) - 1]['cuisine']

In [ ]:
# Task 12.1.2
len(recipes[len(recipes) - 1]['ingredients'])


In [ ]:
# Task 12.1.3
find_id = 17636
for recipe in recipes:
    if recipe['id'] == find_id:
        print(recipe['ingredients'])
        break



In [ ]:
# Task 12.1.4
find_id = 42013
for recipe in recipes:
    if recipe['id'] == find_id:
        print(len(recipe['ingredients']))
        break


In [ ]:
# Task 12.1.5
find_id = 23629
for recipe in recipes:
    if recipe['id'] == find_id:
        print(recipe['ingredients'])
        break

### Группировка данных

In [ ]:
cuisines = []  # создаём пустой список для хранения уникальных значений кухонь
for recipe in recipes:  # начинаем перебор всех рецептов
   if not(recipe['cuisine'] in cuisines):  # если тип кухни текущего блюда ещё не встречался
        cuisines.append(recipe['cuisine']) # добавляем его к списку cuisines
len(cuisines)


In [ ]:
# вариант с использованием множества

cuisines = set()  # создаём пустое множество для хранения уникальных значений кухонь
for recipe in recipes:  # начинаем перебор всех рецептов
   cuisines.add(recipe['cuisine']) # добавляем название типа кухни к множеству
len(cuisines)

In [ ]:
# Задание 12.3.1
# Сколько ингредиентов включено в состав всех блюд, описанных в наборе данных?

product_composition = set()

for recipe in recipes:
    for product in recipe['ingredients']:
        product_composition.add(product)

print(len(product_composition))

In [ ]:
# Задание 12.3.1
# Сколько ингредиентов встречается в рецептах блюд итальянской кухни ('cuisine' = italian)?

italian_product = set()

for recipe in recipes:
    if recipe['cuisine'] == 'italian':
        for product in recipe['ingredients']:
            italian_product.add(product)

print('italian product:', len(italian_product))

In [ ]:
# Задание 12.3.3
# Какие ингредиенты не встречаются в рецептах блюд русской кухни ('cuisine' = 'russian')?

russian_product = set()
check_list = set()
[check_list.add(x) for x in ['bacon', 'bread slices', 'buttermilk', 'red beets', 'mozzarella cheese', 'carrots']]

for recipe in recipes:
    if recipe['cuisine'] == 'russian':
        for product in recipe['ingredients']:
            russian_product.add(product)

check_list - russian_product

In [ ]:
food = collections.Counter()

for recipe in recipes:
    for item in recipe['ingredients']:
        food[item] += 1

food['sugar'], food['eggs']

In [ ]:
# Задание 12.3.4
# Какие ингредиенты входят в состав более чем 20% блюд, представленных в наборе данных?


top_products = set()
tresh_hold = 0.2 * len(recipes)

check_list = ['water', 'olive oil', 'salt', 'garlic', 'pepper']

for item, count in food.items():
    if count > tresh_hold:
        top_products.add(item)

for product in check_list:
    if product in top_products:
        print('[x]', product)
    else:
        print('[ ]', product)


In [ ]:
# Задание 12.3.5
one_recipe = set()
max_freq = 0
top_ingredients = ''

for item, count in food.items():
    if count > max_freq:
        top_ingredients = item
        max_freq = count
    elif count == 1:
        one_recipe.add(item)

top_ingredients

In [ ]:
# Задание 12.3.6
len(one_recipe)

## 12.4 Создание датафрейма

In [ ]:
df = pd.DataFrame(recipes)
df.sample(7)

In [ ]:
df.shape

In [ ]:
# Task 12.4.6

# import collections
# import pandas as pd

def find_item(cell):
    if item in cell:
        return 1
    return 0

# with open('recipes.json') as f:
with open(project_dir + 'recipes.json') as f:
   recipes = json.load(f)

df = pd.DataFrame(recipes)

food = collections.Counter()

for recipe in recipes:
    for item in recipe['ingredients']:
        food[item] += 1

ingredients = food.keys()


for item in ingredients:
    df[item] = df['ingredients'].apply(find_item)


df['ingredients'] = df['ingredients'].apply(lambda x: len(x))


In [ ]:
df.head()

In [ ]:
df.to_csv(project_dir + 'recipes.csv', index = False)

In [ ]:
# df.to_csv(project_dir + '1-recipes.csv', index = True)

# END